In [1]:
import pandas as pd
from datetime import datetime
import numpy as np

In [2]:
pd.set_option('display.max_columns', None)
pd.set_option('display.float_format', lambda x: '%.0f' % x)

In [28]:
# dataset de registros de ocupação de leitos por COVID nos estabelecimentos de saúde do DF
dfsus = pd.read_excel('dados/originais/e-sus-ocupação-DF-excel-v.2.xlsx', dtype={'cnes': 'str'})

# dataset com os dados dos estabelecimentos de Saúde do Brasil
dfestab = pd.read_csv('dados/originais/tbEstabelecimento202102.csv', sep=';', dtype={'CO_CNES': 'str'})

C:\Users\halissonsg\Anaconda3\envs\panel\lib\site-packages\IPython\core\interactiveshell.py:3155: DtypeWarning: Columns (0,36,45) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


### Trabalhando no dataset de ocupação de Leitos

In [29]:
dfsus.head()

,dataNotificacao,cnes,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,saidaSuspeitaObitos,saidaSuspeitaAltas,saidaConfirmadaObitos,saidaConfirmadaAltas,origem,estado,municipio,_created_at,_updated_at
0,2020-04-01T03:00:00.000Z,5717515,257,29,0,0,0,0,0,0,parse-cloud,Distrito Federal,Brasília,2020-08-07T20:17:49.718Z,2020-08-11T17:49:16.386Z
1,2020-04-01T03:00:00.000Z,10456,358,57,0,1,0,0,0,0,parse-cloud,Distrito Federal,Brasília,2020-08-13T17:30:25.254Z,2020-08-13T17:30:25.254Z
2,2020-04-01T03:00:00.000Z,10464,73,13,69,13,0,4,1,2,parse-cloud,Distrito Federal,Brasília,2020-08-17T13:56:07.749Z,2020-08-25T19:53:26.654Z
3,2020-04-01T03:00:00.000Z,7978642,0,16,0,16,0,0,0,1,parse-cloud,Distrito Federal,Brasília,2020-09-01T18:07:49.732Z,2020-09-01T18:07:49.732Z
4,2020-04-02T03:00:00.000Z,5717515,248,29,0,0,0,0,0,0,parse-cloud,Distrito Federal,Brasília,2020-08-07T20:18:21.541Z,2020-08-11T18:03:14.022Z


In [20]:
# função para converter em datetime os compos de data no dataset dfsus
def convert_date(dtvalue):
    return pd.to_datetime(datetime.fromisoformat(dtvalue[:-1]).strftime('%d/%m/%Y %H:%M:%S'), format='%d/%m/%Y %H:%M:%S')

In [30]:
# convertendo os campos do tipo date
dfsus['dataNotificacao'] = dfsus['dataNotificacao'].apply(convert_date)
dfsus['_created_at'] = dfsus['_created_at'].apply(convert_date)
dfsus['_updated_at'] = dfsus['_updated_at'].apply(convert_date)

In [31]:
# verificando a conversão
dfsus.dtypes

dataNotificacao          datetime64[ns]
cnes                             object
ocupacaoSuspeitoCli               int64
ocupacaoSuspeitoUti               int64
ocupacaoConfirmadoCli             int64
ocupacaoConfirmadoUti             int64
saidaSuspeitaObitos               int64
saidaSuspeitaAltas                int64
saidaConfirmadaObitos             int64
saidaConfirmadaAltas              int64
origem                           object
estado                           object
municipio                        object
_created_at              datetime64[ns]
_updated_at              datetime64[ns]
dtype: object

In [32]:
dfsus.head()

,dataNotificacao,cnes,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,saidaSuspeitaObitos,saidaSuspeitaAltas,saidaConfirmadaObitos,saidaConfirmadaAltas,origem,estado,municipio,_created_at,_updated_at
0,2020-04-01 03:00:00,5717515,257,29,0,0,0,0,0,0,parse-cloud,Distrito Federal,Brasília,2020-08-07 20:17:49,2020-08-11 17:49:16
1,2020-04-01 03:00:00,10456,358,57,0,1,0,0,0,0,parse-cloud,Distrito Federal,Brasília,2020-08-13 17:30:25,2020-08-13 17:30:25
2,2020-04-01 03:00:00,10464,73,13,69,13,0,4,1,2,parse-cloud,Distrito Federal,Brasília,2020-08-17 13:56:07,2020-08-25 19:53:26
3,2020-04-01 03:00:00,7978642,0,16,0,16,0,0,0,1,parse-cloud,Distrito Federal,Brasília,2020-09-01 18:07:49,2020-09-01 18:07:49
4,2020-04-02 03:00:00,5717515,248,29,0,0,0,0,0,0,parse-cloud,Distrito Federal,Brasília,2020-08-07 20:18:21,2020-08-11 18:03:14


In [49]:
# adicionando zeros a esquerda nos registros da coluna 'cnes' que possuem menos de 7 digitos
dfsus['cnes'] = dfsus['cnes'].apply(lambda x: x.zfill(7))

In [50]:
dfsus.head()

,dataNotificacao,cnes,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,saidaSuspeitaObitos,saidaSuspeitaAltas,saidaConfirmadaObitos,saidaConfirmadaAltas,origem,estado,municipio,_created_at,_updated_at
0,2020-04-01 03:00:00,5717515,257,29,0,0,0,0,0,0,parse-cloud,Distrito Federal,Brasília,2020-08-07 20:17:49,2020-08-11 17:49:16
1,2020-04-01 03:00:00,0010456,358,57,0,1,0,0,0,0,parse-cloud,Distrito Federal,Brasília,2020-08-13 17:30:25,2020-08-13 17:30:25
2,2020-04-01 03:00:00,0010464,73,13,69,13,0,4,1,2,parse-cloud,Distrito Federal,Brasília,2020-08-17 13:56:07,2020-08-25 19:53:26
3,2020-04-01 03:00:00,7978642,0,16,0,16,0,0,0,1,parse-cloud,Distrito Federal,Brasília,2020-09-01 18:07:49,2020-09-01 18:07:49
4,2020-04-02 03:00:00,5717515,248,29,0,0,0,0,0,0,parse-cloud,Distrito Federal,Brasília,2020-08-07 20:18:21,2020-08-11 18:03:14


In [61]:
dfsus.shape

(4049, 15)

In [62]:
dfsus['cnes'].value_counts(dropna=False)

5717515      366
0010456      362
0010464      308
6730914      280
0010510      272
7978642      265
0010472      265
0252492      244
2672197      239
0010502      239
0010545      231
3276678      201
0010480      170
3018520      149
0174971      147
3019616      119
0010510\t     86
6243495       58
2645157       33
0010529       14
9699848        1
Name: cnes, dtype: int64

In [67]:
dfsus.loc[dfsus['cnes'] == '0010510\t','cnes'] = '0010510'

In [68]:
dfsus['cnes'].value_counts(dropna=False)

5717515    366
0010456    362
0010510    358
0010464    308
6730914    280
7978642    265
0010472    265
0252492    244
2672197    239
0010502    239
0010545    231
3276678    201
0010480    170
3018520    149
0174971    147
3019616    119
6243495     58
2645157     33
0010529     14
9699848      1
Name: cnes, dtype: int64

### Trabalhando no dataset de Estabelecimentos

In [33]:
dfestab.head()

,CO_UNIDADE,CO_CNES,NU_CNPJ_MANTENEDORA,TP_PFPJ,NIVEL_DEP,NO_RAZAO_SOCIAL,NO_FANTASIA,NO_LOGRADOURO,NU_ENDERECO,NO_COMPLEMENTO,NO_BAIRRO,CO_CEP,CO_REGIAO_SAUDE,CO_MICRO_REGIAO,CO_DISTRITO_SANITARIO,CO_DISTRITO_ADMINISTRATIVO,NU_TELEFONE,NU_FAX,NO_EMAIL,NU_CPF,NU_CNPJ,CO_ATIVIDADE,CO_CLIENTELA,NU_ALVARA,DT_EXPEDICAO,TP_ORGAO_EXPEDIDOR,DT_VAL_LIC_SANI,TP_LIC_SANI,TP_UNIDADE,CO_TURNO_ATENDIMENTO,CO_ESTADO_GESTOR,CO_MUNICIPIO_GESTOR,"TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')",CO_USUARIO,CO_CPFDIRETORCLN,REG_DIRETORCLN,ST_ADESAO_FILANTROP,CO_MOTIVO_DESAB,NO_URL,NU_LATITUDE,NU_LONGITUDE,"TO_CHAR(DT_ATU_GEO,'DD/MM/YYYY')",NO_USUARIO_GEO,CO_NATUREZA_JUR,TP_ESTAB_SEMPRE_ABERTO,ST_GERACREDITO_GERENTE_SGIF,ST_CONEXAO_INTERNET,CO_TIPO_UNIDADE,NO_FANTASIA_ABREV,TP_GESTAO,"TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY')",CO_TIPO_ESTABELECIMENTO,CO_ATIVIDADE_PRINCIPAL,ST_CONTRATO_FORMALIZADO
0,MG0000311800000022588461000160,2139006,NaN,3,1,FUNDACAO MUNICIPAL DE SAUD DE CONGONHAS,POSTO DE SAUDE PLATAFORMA,VILA PLATAFORMA,S/N,NaN,VILA MARQUES,36404000,03,NaN,NaN,NaN,(31)37371950,NaN,NaN,NaN,22588461000160,4,1,157/03/2000,06-dez-2000 00:00:00,1,NaN,NaN,1,3,31,311800,16/02/2002,silvia,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN,NaN,1000,NaN,NaN,NaN,NaN,NaN,M,12/03/2002,NaN,NaN,NaN
1,GO00000000000000000000000025747,2342103,1740455000106,3,3,PREFEITURA MUNICIPAL DE ALTO PARAISO,POSTO DE SAUDE DA FRATERNIDADE,CIDADE DA FRATERNIDADE,S/N,NaN,ZONA RURAL,73770000,2,NaN,NaN,NaN,(61)6461335,(61)6461450,NaN,NaN,NaN,4,3,NaN,NaN,NaN,NaN,NaN,1,3,52,520060,25/04/2003,relton,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN,NaN,1000,NaN,NaN,NaN,NaN,NaN,M,30/10/2001,NaN,NaN,NaN
2,GO00000000000000000000000037184,2342146,1740455000106,3,3,PREFEITURA MUNICIPAL DE ALTO PARAISO,CENTRO DE ASSISTENCIA A SAUDE BUCAL DO ESCOLAR...,RUA PALIPALA,04,QDA 18,PARAISO VELHO,73770000,2,NaN,NaN,NaN,(61)6461103,(61)6461450,NaN,NaN,NaN,4,3,NaN,NaN,NaN,NaN,NaN,22,3,52,520060,25/04/2003,relton,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN,NaN,1000,NaN,NaN,NaN,NaN,NaN,M,30/10/2001,NaN,NaN,NaN
3,2311302565285,2565285,23444748000189,3,3,PREFEITURA MUNICIPAL DE QUIXADA,POSTO DE SAUDE NICODEMOS PEREIRA DE SOUSA,LAGOA NOVA,S/N,ZONA RURAL,LAGOA NOVA,63900000,0008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,1,191/2001,30-abr-2001 00:00:00,2,NaN,NaN,1,2,23,231130,08/12/2001,selene,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN,NaN,1000,NaN,NaN,NaN,NaN,NaN,M,08/05/2002,NaN,NaN,NaN
4,2311302565307,2565307,23444748000189,3,3,PREFEITURA MUNICIPAL DE QUIXADA,POSTO DE SAUDE DE LAGOA DA PEDRA,VILA DA LAGOA DA PEDRA,S/N,ZON RURAL,VARZEA DA ONCA,63900000,0008,NaN,NaN,NaN,(88)4121109,NaN,NaN,NaN,NaN,4,1,18201,30-abr-2001 00:00:00,2,NaN,NaN,1,2,23,231130,08/12/2001,selene,NaN,NaN,NaN,8,NaN,NaN,NaN,NaN,NaN,1000,NaN,NaN,NaN,NaN,NaN,M,08/05/2002,NaN,NaN,NaN


In [34]:
# excluindo os registros sem latitude/longitude no dataset de estabelecimentos
dfestab.dropna(subset=['NU_LATITUDE', 'NU_LONGITUDE'], inplace=True)

In [66]:
dfestab[dfestab['CO_ESTADO_GESTOR'] == 53]

,CO_UNIDADE,CO_CNES,NU_CNPJ_MANTENEDORA,TP_PFPJ,NIVEL_DEP,NO_RAZAO_SOCIAL,NO_FANTASIA,NO_LOGRADOURO,NU_ENDERECO,NO_COMPLEMENTO,NO_BAIRRO,CO_CEP,CO_REGIAO_SAUDE,CO_MICRO_REGIAO,CO_DISTRITO_SANITARIO,CO_DISTRITO_ADMINISTRATIVO,NU_TELEFONE,NU_FAX,NO_EMAIL,NU_CPF,NU_CNPJ,CO_ATIVIDADE,CO_CLIENTELA,NU_ALVARA,DT_EXPEDICAO,TP_ORGAO_EXPEDIDOR,DT_VAL_LIC_SANI,TP_LIC_SANI,TP_UNIDADE,CO_TURNO_ATENDIMENTO,CO_ESTADO_GESTOR,CO_MUNICIPIO_GESTOR,"TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')",CO_USUARIO,CO_CPFDIRETORCLN,REG_DIRETORCLN,ST_ADESAO_FILANTROP,CO_MOTIVO_DESAB,NO_URL,NU_LATITUDE,NU_LONGITUDE,"TO_CHAR(DT_ATU_GEO,'DD/MM/YYYY')",NO_USUARIO_GEO,CO_NATUREZA_JUR,TP_ESTAB_SEMPRE_ABERTO,ST_GERACREDITO_GERENTE_SGIF,ST_CONEXAO_INTERNET,CO_TIPO_UNIDADE,NO_FANTASIA_ABREV,TP_GESTAO,"TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY')",CO_TIPO_ESTABELECIMENTO,CO_ATIVIDADE_PRINCIPAL,ST_CONTRATO_FORMALIZADO
191,5300109428208,9428208,NaN,3,1,LABORATORIO SABIN DE ANALISES CLINICAS S A,SABIN MEDICINA DIAGNOSTICA SOBRADINHO QD 09,Q 9 COMERCIO LOCAL,04,LOJA 01 E 02,SOBRADINHO,73036610,NaN,NaN,0170,NaN,61-3329-8000,NaN,atossocietarios@sabin.com.br,NaN,718528012611,4,1,LAB-01167-09,26-jun-2020 00:00:00,1,26-jun-2021 00:00:00,1,39,3,53,530010,09/10/2020,GECAD,1218545666,NaN,NaN,NaN,NaN,-15.642611,-47.803874,01/08/2019,DALVANSABIN,2054,N,NaN,S,NaN,NaN,E,23/02/2018,18,2,N
216,5300109431403,9431403,NaN,3,1,FLORENCE DERMATOLOGIA MEHLBERG EIRELI,DR DANIEL MEHLBERG DERMATOLOGIA,RUA 5 NORTE,03,SALA 308,NORTE AGUAS CLARAS,71907720,DF,NaN,9901,NaN,61 30461536,NaN,jorgefilho@jccontabil.com,NaN,21383506000105,4,1,53600069234,24-jan-2018 00:00:00,1,19-jan-2019 00:00:00,1,22,3,53,530010,24/10/2018,GECAD,70210900130,NaN,NaN,NaN,NaN,-15.78,-47.93,27/09/2018,GABRIEL,2305,N,NaN,S,NaN,NaN,E,23/02/2018,16,1,NaN
227,5300109212388,9212388,NaN,3,1,NUTRINDO O CONHECIMENTO EIRELI,NUTRINDO O CONHECIMENTO,SEPS 710 910,S/N,CONJ D,ASA SUL,70390108,NaN,NaN,NaN,NaN,61 96731680,NaN,NaN,NaN,10885562000113,4,3,RLE,07-jun-2019 00:00:00,1,05-jun-2022 00:00:00,1,22,3,53,530010,19/07/2019,GECAD,4400670651,NaN,NaN,NaN,NaN,-15.81277545,-47.9116720441306,18/07/2019,RIELISSONBM,2305,N,NaN,S,NaN,NaN,E,09/05/2017,16,1,N
429,5300109808876,9808876,NaN,3,1,RITA VILANOVA SERVICOS MEDICOS EIRELI,RITA VILANOVA SERVICOS MEDICOS,ST SEPS 709 909 CONJUNTO A BLOCO B SALA,203,PARTE A,ASA SUL,70390095,NaN,NaN,0015,NaN,32978444,NaN,CENTRALMINAS-CONTABIL@HOTMAIL.COM,NaN,33246868000125,4,3,EAS.56409-01,14-ago-2020 00:00:00,1,14-ago-2021 00:00:00,1,36,3,53,530010,07/10/2020,GECAD,85838950153,NaN,NaN,NaN,NaN,-15.78,-47.93,06/10/2020,CENTRAL,2062,N,NaN,S,NaN,NaN,E,02/06/2019,16,1,N
649,5300109839046,9839046,NaN,3,1,SAUDE SIM LTDA,DR SIM 0004,QNM 17 CONJUNTO H LOTE 30 SALAS 401 402,403,NaN,CEILANDIA,72215178,NaN,NaN,NaN,NaN,613797952,NaN,unicondf@gmail.com,NaN,2464179000406,4,1,QR CODE,26-jun-2019 00:00:00,1,04-set-2021 00:00:00,2,22,3,53,530010,02/07/2019,GECAD,70529531100,NaN,NaN,NaN,NaN,-15.815470631782516,-48.097500801086426,27/06/2019,HELIO,2062,N,NaN,S,NaN,NaN,E,05/07/2019,16,1,N
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423930,5300106691935,6691935,NaN,3,1,CAPTAMED CUIDADOS CONTINUADOS LTDA,HMB SAUDE BRASILIA,SMHN QUADRA 2 BLOCO C,44,SALA 301 706 707 71,ASA NORTE,70710149,NaN,NaN,0010,NaN,31 30481122,NaN,financeiro@captamed.com.br,NaN,5503624000226,4,NaN,EAS-0036-02,30-mar-2020 00:00:00,1,30-mar-2021 00:00:00,1,77,6,53,530010,06/08/2020,GECAD,5730294662,NaN,NaN,NaN,NaN,-15.784169815793199,-47.884801626205444,30/01/2019,HMB05,2062,S,NaN,S,NaN,NaN,E,06/02/2011,12,6,NaN
423937,5300106665993,6665993,NaN,3,1,BABY PED PRONTO ATENDIMENTO INFANTIL LTDA,BABY PED PRONTO ATENDIMENTO INFANTIL,SHIS QI 07 BLOCO F SALA,02,NaN,LAGO SUL,71615660,NaN,NaN,0090,NaN,35468005,NaN,baby.ped@hotmail.com,NaN,11092950000100,4,NaN,EAS.00091-03,07-mai-2020 00:00:00,1,07-mai-2021 00:

In [55]:
# verificando o tamanho do dataset após a exclusão dos registros que não possuem latitude/longitude
dfestab.shape

(352813, 54)

In [51]:
# adicionando zeros a esquerda nos registros da coluna 'CO_CNES' que possuem menos de 7 digitos
dfestab['CO_CNES'] = dfestab['CO_CNES'].apply(lambda x: x.zfill(7))

In [54]:
# verificando se os registros de CO_CNES são únicos no dataframe
assert(dfestab['CO_CNES'].value_counts(dropna=False).all() == 1)

### Tabalhando no dataframe combinado entre os dois (Chave -> CNES)

In [69]:
# juntando os 02 dataframes
dfj = dfsus.merge(dfestab, how='inner', left_on='cnes', right_on='CO_CNES')
dfj.head()

,dataNotificacao,cnes,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,saidaSuspeitaObitos,saidaSuspeitaAltas,saidaConfirmadaObitos,saidaConfirmadaAltas,origem,estado,municipio,_created_at,_updated_at,CO_UNIDADE,CO_CNES,NU_CNPJ_MANTENEDORA,TP_PFPJ,NIVEL_DEP,NO_RAZAO_SOCIAL,NO_FANTASIA,NO_LOGRADOURO,NU_ENDERECO,NO_COMPLEMENTO,NO_BAIRRO,CO_CEP,CO_REGIAO_SAUDE,CO_MICRO_REGIAO,CO_DISTRITO_SANITARIO,CO_DISTRITO_ADMINISTRATIVO,NU_TELEFONE,NU_FAX,NO_EMAIL,NU_CPF,NU_CNPJ,CO_ATIVIDADE,CO_CLIENTELA,NU_ALVARA,DT_EXPEDICAO,TP_ORGAO_EXPEDIDOR,DT_VAL_LIC_SANI,TP_LIC_SANI,TP_UNIDADE,CO_TURNO_ATENDIMENTO,CO_ESTADO_GESTOR,CO_MUNICIPIO_GESTOR,"TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')",CO_USUARIO,CO_CPFDIRETORCLN,REG_DIRETORCLN,ST_ADESAO_FILANTROP,CO_MOTIVO_DESAB,NO_URL,NU_LATITUDE,NU_LONGITUDE,"TO_CHAR(DT_ATU_GEO,'DD/MM/YYYY')",NO_USUARIO_GEO,CO_NATUREZA_JUR,TP_ESTAB_SEMPRE_ABERTO,ST_GERACREDITO_GERENTE_SGIF,ST_CONEXAO_INTERNET,CO_TIPO_UNIDADE,NO_FANTASIA_ABREV,TP_GESTAO,"TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY')",CO_TIPO_ESTABELECIMENTO,CO_ATIVIDADE_PRINCIPAL,ST_CONTRATO_FORMALIZADO
0,2020-04-01 03:00:00,5717515,257,29,0,0,0,0,0,0,parse-cloud,Distrito Federal,Brasília,2020-08-07 20:17:49,2020-08-11 17:49:16,5300105717515,5717515,NaN,3,1,HOSPITAL REGIONAL DE SANTA MARIA,HRSM,AC 102,S/N,BLOCOS A B C D,SANTA MARIA,72502100,03,NaN,0150,NaN,33151265,NaN,nuprochrsm@gmail.com,NaN,28481233000920,4,3,NaN,21-jan-2019 00:00:00,NaN,01-jan-2020 00:00:00,1,5,6,53,530010,04/03/2021,GECAD,69354227104,24253,2,NaN,NaN,-16.34766787476504,-48.882677257061005,19/05/2020,NUCAIS,3077,S,NaN,S,NaN,NaN,E,09/04/2008,6,9,S
1,2020-04-02 03:00:00,5717515,248,29,0,0,0,0,0,0,parse-cloud,Distrito Federal,Brasília,2020-08-07 20:18:21,2020-08-11 18:03:14,5300105717515,5717515,NaN,3,1,HOSPITAL REGIONAL DE SANTA MARIA,HRSM,AC 102,S/N,BLOCOS A B C D,SANTA MARIA,72502100,03,NaN,0150,NaN,33151265,NaN,nuprochrsm@gmail.com,NaN,28481233000920,4,3,NaN,21-jan-2019 00:00:00,NaN,01-jan-2020 00:00:00,1,5,6,53,530010,04/03/2021,GECAD,69354227104,24253,2,NaN,NaN,-16.34766787476504,-48.882677257061005,19/05/2020,NUCAIS,3077,S,NaN,S,NaN,NaN,E,09/04/2008,6,9,S
2,2020-04-03 03:00:00,5717515,246,29,0,0,0,0,0,0,parse-cloud,Distrito Federal,Brasília,2020-08-07 20:18:54,2020-08-11 18:03:42,5300105717515,5717515,NaN,3,1,HOSPITAL REGIONAL DE SANTA MARIA,HRSM,AC 102,S/N,BLOCOS A B C D,SANTA MARIA,72502100,03,NaN,0150,NaN,33151265,NaN,nuprochrsm@gmail.com,NaN,28481233000920,4,3,NaN,21-jan-2019 00:00:00,NaN,01-jan-2020 00:00:00,1,5,6,53,530010,04/03/2021,GECAD,69354227104,24253,2,NaN,NaN,-16.34766787476504,-48.882677257061005,19/05/2020,NUCAIS,3077,S,NaN,S,NaN,NaN,E,09/04/2008,6,9,S
3,2020-04-04 03:00:00,5717515,251,29,0,0,0,0,0,0,parse-cloud,Distrito Federal,Brasília,2020-08-11 18:05:05,2020-08-11 18:05:05,5300105717515,5717515,NaN,3,1,HOSPITAL REGIONAL DE SANTA MARIA,HRSM,AC 102,S/N,BLOCOS A B C D,SANTA MARIA,72502100,03,NaN,0150,NaN,33151265,NaN,nuprochrsm@gmail.com,NaN,28481233000920,4,3,NaN,21-jan-2019 00:00:00,NaN,01-jan-2020 00:00:00,1,5,6,53,530010,04/03/2021,GECAD,69354227104,24253,2,NaN,NaN,-16.34766787476504,-48.882677257061005,19/05/2020,NUCAIS,3077,S,NaN,S,NaN,NaN,E,09/04/2008,6,9,S
4,2020-04-05 03:00:00,5717515,253,24,0,0,0,0,0,0,parse-cloud,Distrito Federal,Brasília,2020-08-07 20:19:18,2020-08-11 18:06:31,5300105717515,5717515,NaN,3,1,HOSPITAL REGIONAL DE SANTA MARIA,HRSM,AC 102,S/N,BLOCOS A B C D,SANTA MARIA,72502100,03,NaN,0150,NaN,33151265,NaN,nuprochrsm@gmail.com,NaN,28481233000920,4,3,NaN,21-jan-2019 00:00:00,NaN,01-jan-2020 00:00:00,1,5,6,53,530010,04/03/2021,GECAD,69354227104,24253,2,NaN,NaN,-16.34766787476504,-48.882677257061005,19/05/2020,NUCAIS,3077,S,NaN,S,NaN,NaN,E,09/04/2008,6,9,S


In [70]:
dfj.shape

(4049, 69)

In [73]:
dfj.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4049 entries, 0 to 4048
Data columns (total 69 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   dataNotificacao                              4049 non-null   datetime64[ns]
 1   cnes                                         4049 non-null   object        
 2   ocupacaoSuspeitoCli                          4049 non-null   int64         
 3   ocupacaoSuspeitoUti                          4049 non-null   int64         
 4   ocupacaoConfirmadoCli                        4049 non-null   int64         
 5   ocupacaoConfirmadoUti                        4049 non-null   int64         
 6   saidaSuspeitaObitos                          4049 non-null   int64         
 7   saidaSuspeitaAltas                           4049 non-null   int64         
 8   saidaConfirmadaObitos                        4049 non-null   int64         
 9

In [75]:
pd.reset_option('display.float_format')

In [80]:
dfj = dfj.astype({'NU_LATITUDE': 'float64', 'NU_LONGITUDE': 'float64'}, copy=False)

In [81]:
dfj.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 4049 entries, 0 to 4048
Data columns (total 69 columns):
 #   Column                                       Non-Null Count  Dtype         
---  ------                                       --------------  -----         
 0   dataNotificacao                              4049 non-null   datetime64[ns]
 1   cnes                                         4049 non-null   object        
 2   ocupacaoSuspeitoCli                          4049 non-null   int64         
 3   ocupacaoSuspeitoUti                          4049 non-null   int64         
 4   ocupacaoConfirmadoCli                        4049 non-null   int64         
 5   ocupacaoConfirmadoUti                        4049 non-null   int64         
 6   saidaSuspeitaObitos                          4049 non-null   int64         
 7   saidaSuspeitaAltas                           4049 non-null   int64         
 8   saidaConfirmadaObitos                        4049 non-null   int64         
 9

In [82]:
dfj.to_csv('dados/processados/e-sus-ocupacao-estabelecimento.csv', index=False)

In [3]:
dfj = pd.read_csv('dados/processados/e-sus-ocupacao-estabelecimento.csv', parse_dates=['dataNotificacao'])

In [23]:
# Agregações para mostrar no dashboard
lst_columns = ['dataNotificacao', 'ocupacaoSuspeitoCli', 'saidaConfirmadaObitos', 'NO_FANTASIA', 
               'NU_LATITUDE', 'NU_LONGITUDE']

df_tabela = dfj[lst_columns].groupby([
    dfj['dataNotificacao'].dt.to_period('M'),
    'NO_FANTASIA',
    'NU_LATITUDE', 
    'NU_LONGITUDE'    
]).agg({
    'ocupacaoSuspeitoCli': 'mean',
    'saidaConfirmadaObitos': 'sum'
       }).reset_index()

df_tabela

,dataNotificacao,NO_FANTASIA,NU_LATITUDE,NU_LONGITUDE,ocupacaoSuspeitoCli,saidaConfirmadaObitos
0,2020-04,HOSPITAL DAHER LAGO SUL SA,-16,-48,0,7
1,2020-04,HOSPITAL DE BASE DO DISTRITO FEDERAL,-16,-48,389,0
2,2020-04,HOSPITAL UNIVERSITARIO DE BRASILIA,-16,-48,40,0
3,2020-04,HRAN,-16,-48,45,10
4,2020-04,HRSM,-16,-49,252,1
...,...,...,...,...,...,...
148,2021-03,HRBZ,-16,-48,84,1
149,2021-03,HRG,-16,-48,248,21
150,2021-03,HRS,-16,-48,32,21
151,2021-03,HRSAM HOSPITAL REGIONAL DE SAMAMBAIA,-16,-48,42,23


In [33]:
df_tabela.dtypes

dataNotificacao          period[M]
NO_FANTASIA                 object
NU_LATITUDE                float64
NU_LONGITUDE               float64
ocupacaoSuspeitoCli        float64
saidaConfirmadaObitos        int64
dtype: object

In [37]:
df_tabela['dataNotificacao'].dtype not in ['object', 'int64', 'float64']

False

In [21]:
{linha: [df_tabela.iloc[linha][col] for col in df_tabela.columns] for linha in range(len(df_tabela))}

{0: ['04-2020',
  'HOSPITAL DAHER LAGO SUL SA',
  -15.841719528091218,
  -47.885756492614746,
  0.5,
  7],
 1: ['04-2020',
  'HOSPITAL DE BASE DO DISTRITO FEDERAL',
  -15.800653,
  -47.888283,
  389.06666666666666,
  0],
 2: ['04-2020',
  'HOSPITAL UNIVERSITARIO DE BRASILIA',
  -15.77175964443316,
  -47.87372410297394,
  39.80952380952381,
  0],
 3: ['04-2020', 'HRAN', -15.7857462, -47.8827625, 44.7, 10],
 4: ['04-2020',
  'HRSM',
  -16.34766787476504,
  -48.882677257061005,
  252.23333333333332,
  1],
 5: ['05-2020',
  'HOSPITAL DAHER LAGO SUL SA',
  -15.841719528091218,
  -47.885756492614746,
  0.0,
  10],
 6: ['05-2020',
  'HOSPITAL DE BASE DO DISTRITO FEDERAL',
  -15.800653,
  -47.888283,
  378.64516129032256,
  0],
 7: ['05-2020',
  'HOSPITAL DE CAMPANHA COVID 19',
  -15.782611,
  -47.899211,
  19.615384615384617,
  0],
 8: ['05-2020',
  'HOSPITAL UNIVERSITARIO DE BRASILIA',
  -15.77175964443316,
  -47.87372410297394,
  39.42424242424242,
  5],
 9: ['05-2020', 'HRAN', -15.7857462,

In [7]:
df_mapa = df_tabela.groupby(['NO_FANTASIA', 'NU_LATITUDE', 'NU_LONGITUDE']).agg({
    'ocupacaoSuspeitoCli': 'mean',
    'saidaConfirmadaObitos': 'sum'
}).reset_index().round(2)
df_mapa

,NO_FANTASIA,NU_LATITUDE,NU_LONGITUDE,ocupacaoSuspeitoCli,saidaConfirmadaObitos
0,HOME HOSPITAL ORTOPEDICO E MEDICINA ESPECIALIZADA,-16,-48,35,36
1,HOSPITAL DAHER LAGO SUL SA,-16,-48,0,200
2,HOSPITAL DE BASE DO DISTRITO FEDERAL,-16,-48,462,475
3,HOSPITAL DE CAMPANHA COVID 19,-16,-48,82,30
4,HOSPITAL DE CAMPANHA DO CENTRO MEDICO DA PM,-16,-48,11,357
5,HOSPITAL REGIONAL DE CEILANDIA,-16,-48,118,94
6,HOSPITAL SANTA LUCIA NORTE,-16,-48,0,0
7,HOSPITAL SAO FRANCISCO,-16,-48,0,20
8,HOSPITAL SAO MATEUS,-16,-48,0,186
9,HOSPITAL UNIVERSITARIO DE BRASILIA,-16,-48,45,76


In [94]:
# media mensal de ocupacao clinica dos leitos
dfj['dataNotificacao'].dt.to_period('M').min().strftime('%m-%Y')

'04-2020'

In [95]:
dfj['dataNotificacao'].dt.to_period('M').max().strftime('%m-%Y')

'03-2021'

In [120]:
dfj['dataNotificacao'].dt.to_period('M').nunique()

12

In [157]:
# {'label': '100°C', 'style': {'color': '#f50'}}
periodos_range = {i: {'label': label.strftime('%m-%Y'), 'style':{'transform': 'rotate(-40deg)'}} for i,label in enumerate(df_tabela['dataNotificacao'].unique())}
periodos_range

{0: {'label': '04-2020', 'style': {'transform': 'rotate(-40deg)'}},
 1: {'label': '05-2020', 'style': {'transform': 'rotate(-40deg)'}},
 2: {'label': '06-2020', 'style': {'transform': 'rotate(-40deg)'}},
 3: {'label': '07-2020', 'style': {'transform': 'rotate(-40deg)'}},
 4: {'label': '08-2020', 'style': {'transform': 'rotate(-40deg)'}},
 5: {'label': '09-2020', 'style': {'transform': 'rotate(-40deg)'}},
 6: {'label': '10-2020', 'style': {'transform': 'rotate(-40deg)'}},
 7: {'label': '11-2020', 'style': {'transform': 'rotate(-40deg)'}},
 8: {'label': '12-2020', 'style': {'transform': 'rotate(-40deg)'}},
 9: {'label': '01-2021', 'style': {'transform': 'rotate(-40deg)'}},
 10: {'label': '02-2021', 'style': {'transform': 'rotate(-40deg)'}},
 11: {'label': '03-2021', 'style': {'transform': 'rotate(-40deg)'}}}

In [149]:
[*periodos_range]

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]

In [142]:
max(periodos_range)

11

In [138]:
dfj.loc[(dfj['dataNotificacao'].dt.to_period('M') >= periodos_range[0]['label']) & (dfj['dataNotificacao'].dt.to_period('M') <= periodos_range[5]['label'])]

,dataNotificacao,cnes,ocupacaoSuspeitoCli,ocupacaoSuspeitoUti,ocupacaoConfirmadoCli,ocupacaoConfirmadoUti,saidaSuspeitaObitos,saidaSuspeitaAltas,saidaConfirmadaObitos,saidaConfirmadaAltas,origem,estado,municipio,_created_at,_updated_at,CO_UNIDADE,CO_CNES,NU_CNPJ_MANTENEDORA,TP_PFPJ,NIVEL_DEP,NO_RAZAO_SOCIAL,NO_FANTASIA,NO_LOGRADOURO,NU_ENDERECO,NO_COMPLEMENTO,NO_BAIRRO,CO_CEP,CO_REGIAO_SAUDE,CO_MICRO_REGIAO,CO_DISTRITO_SANITARIO,CO_DISTRITO_ADMINISTRATIVO,NU_TELEFONE,NU_FAX,NO_EMAIL,NU_CPF,NU_CNPJ,CO_ATIVIDADE,CO_CLIENTELA,NU_ALVARA,DT_EXPEDICAO,TP_ORGAO_EXPEDIDOR,DT_VAL_LIC_SANI,TP_LIC_SANI,TP_UNIDADE,CO_TURNO_ATENDIMENTO,CO_ESTADO_GESTOR,CO_MUNICIPIO_GESTOR,"TO_CHAR(DT_ATUALIZACAO,'DD/MM/YYYY')",CO_USUARIO,CO_CPFDIRETORCLN,REG_DIRETORCLN,ST_ADESAO_FILANTROP,CO_MOTIVO_DESAB,NO_URL,NU_LATITUDE,NU_LONGITUDE,"TO_CHAR(DT_ATU_GEO,'DD/MM/YYYY')",NO_USUARIO_GEO,CO_NATUREZA_JUR,TP_ESTAB_SEMPRE_ABERTO,ST_GERACREDITO_GERENTE_SGIF,ST_CONEXAO_INTERNET,CO_TIPO_UNIDADE,NO_FANTASIA_ABREV,TP_GESTAO,"TO_CHAR(DT_ATUALIZACAO_ORIGEM,'DD/MM/YYYY')",CO_TIPO_ESTABELECIMENTO,CO_ATIVIDADE_PRINCIPAL,ST_CONTRATO_FORMALIZADO
0,2020-04-01 03:00:00,5717515,257,29,0,0,0,0,0,0,parse-cloud,Distrito Federal,Brasília,2020-08-07 20:17:49,2020-08-11 17:49:16,5300105717515,5717515,NaN,3,1,HOSPITAL REGIONAL DE SANTA MARIA,HRSM,AC 102,S/N,BLOCOS A B C D,SANTA MARIA,72502100,03,NaN,0150,NaN,33151265,NaN,nuprochrsm@gmail.com,NaN,2.848123e+13,4,3.0,NaN,21-jan-2019 00:00:00,NaN,01-jan-2020 00:00:00,1.0,5,6.0,53,530010,04/03/2021,GECAD,6.935423e+10,24253,2.0,NaN,NaN,-16.347668,-48.882677,19/05/2020,NUCAIS,3077.0,S,NaN,S,NaN,NaN,E,09/04/2008,6.0,9.0,S
1,2020-04-02 03:00:00,5717515,248,29,0,0,0,0,0,0,parse-cloud,Distrito Federal,Brasília,2020-08-07 20:18:21,2020-08-11 18:03:14,5300105717515,5717515,NaN,3,1,HOSPITAL REGIONAL DE SANTA MARIA,HRSM,AC 102,S/N,BLOCOS A B C D,SANTA MARIA,72502100,03,NaN,0150,NaN,33151265,NaN,nuprochrsm@gmail.com,NaN,2.848123e+13,4,3.0,NaN,21-jan-2019 00:00:00,NaN,01-jan-2020 00:00:00,1.0,5,6.0,53,530010,04/03/2021,GECAD,6.935423e+10,24253,2.0,NaN,NaN,-16.347668,-48.882677,19/05/2020,NUCAIS,3077.0,S,NaN,S,NaN,NaN,E,09/04/2008,6.0,9.0,S
2,2020-04-03 03:00:00,5717515,246,29,0,0,0,0,0,0,parse-cloud,Distrito Federal,Brasília,2020-08-07 20:18:54,2020-08-11 18:03:42,5300105717515,5717515,NaN,3,1,HOSPITAL REGIONAL DE SANTA MARIA,HRSM,AC 102,S/N,BLOCOS A B C D,SANTA MARIA,72502100,03,NaN,0150,NaN,33151265,NaN,nuprochrsm@gmail.com,NaN,2.848123e+13,4,3.0,NaN,21-jan-2019 00:00:00,NaN,01-jan-2020 00:00:00,1.0,5,6.0,53,530010,04/03/2021,GECAD,6.935423e+10,24253,2.0,NaN,NaN,-16.347668,-48.882677,19/05/2020,NUCAIS,3077.0,S,NaN,S,NaN,NaN,E,09/04/2008,6.0,9.0,S
3,2020-04-04 03:00:00,5717515,251,29,0,0,0,0,0,0,parse-cloud,Distrito Federal,Brasília,2020-08-11 18:05:05,2020-08-11 18:05:05,5300105717515,5717515,NaN,3,1,HOSPITAL REGIONAL DE SANTA MARIA,HRSM,AC 102,S/N,BLOCOS A B C D,SANTA MARIA,72502100,03,NaN,0150,NaN,33151265,NaN,nuprochrsm@gmail.com,NaN,2.848123e+13,4,3.0,NaN,21-jan-2019 00:00:00,NaN,01-jan-2020 00:00:00,1.0,5,6.0,53,530010,04/03/2021,GECAD,6.935423e+10,24253,2.0,NaN,NaN,-16.347668,-48.882677,19/05/2020,NUCAIS,3077.0,S,NaN,S,NaN,NaN,E,09/04/2008,6.0,9.0,S
4,2020-04-05 03:00:00,5717515,253,24,0,0,0,0,0,0,parse-cloud,Distrito Federal,Brasília,2020-08-07 20:19:18,2020-08-11 18:06:31,5300105717515,5717515,NaN,3,1,HOSPITAL REGIONAL DE SANTA MARIA,HRSM,AC 102,S/N,BLOCOS A B C D,SANTA MARIA,72502100,03,NaN,0150,NaN,33151265,NaN,nuprochrsm@gmail.com,NaN,2.848123e+13,4,3.0,NaN,21-jan-2019 00:00:00,NaN,01-jan-2020 00:00:00,1.0,5,6.0,53,530010,04/03/2021,GECAD,6.935423e+10,24253,2.0,NaN,NaN,-16.347668,-48.882677,19/05/2020,NUCAIS,3077.0,S,NaN,S,NaN,NaN,E,09/04/2008,6.0,9.0,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3986,2020

In [100]:
dfj['dataNotificacao'].dt.date.max()

datetime.date(2021, 3, 31)

In [150]:
dfj_filtrado = df_mapa.loc[(df_mapa['dataNotificacao'] >= periodos_range[0]['label']) & (df_mapa['dataNotificacao'] <= periodos_range[5]['label'])]
dfj_filtrado

,dataNotificacao,NO_FANTASIA,NU_LATITUDE,NU_LONGITUDE,ocupacaoSuspeitoCli,saidaConfirmadaObitos
0,2020-04,HOSPITAL DAHER LAGO SUL SA,-15.841720,-47.885756,0.500000,7
1,2020-04,HOSPITAL DE BASE DO DISTRITO FEDERAL,-15.800653,-47.888283,389.066667,0
2,2020-04,HOSPITAL UNIVERSITARIO DE BRASILIA,-15.771760,-47.873724,39.809524,0
3,2020-04,HRAN,-15.785746,-47.882762,44.700000,10
4,2020-04,HRSM,-16.347668,-48.882677,252.233333,1
...,...,...,...,...,...,...
63,2020-09,HRS,-15.647999,-47.792244,40.000000,6
64,2020-09,HRSAM HOSPITAL REGIONAL DE SAMAMBAIA,-15.781640,-47.931193,28.896552,8
65,2020-09,HRSM,-16.347668,-48.882677,267.566667,60
66,2020-09,INSTITUTO DE CARDIOLOGIA DO DISTRITO FEDERAL,-15.802208,-47.935607,9.000000,2
